In [83]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import random

import string
import re
cmap = plt.cm.get_cmap('coolwarm')
import warnings
warnings.filterwarnings('ignore')



# Use for pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import cohen_kappa_score, f1_score

from nltk.tokenize import word_tokenize
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier 
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv


In [86]:
!pip install torch

In [87]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

ValueError: module functions cannot set METH_CLASS or METH_STATIC

In [64]:
df_train = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
df_test = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')

In [65]:
essay_id_dropped = df_train['essay_id']
df_train = df_train.drop('essay_id', axis = 1)


In [66]:
df_train['word_count'] = df_train['full_text'].apply(lambda x: len(x.split()))
df_test['word_count'] = df_test['full_text'].apply(lambda x: len(x.split()))

In [67]:
df_train['essay_length'] = df_train['full_text'].str.len()
df_test['essay_length'] = df_test['full_text'].str.len()

In [68]:
df_train['sentences_count'] = df_train['full_text'].str.count(r'\.')
df_test['sentences_count'] = df_test['full_text'].str.count(r'\.')

In [69]:
df_train['paragraph_count'] = df_train['full_text'].str.count(r'\n') + 1
df_test['paragraph_count'] = df_test['full_text'].str.count(r'\n') + 1

In [70]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   full_text        17307 non-null  object
 1   score            17307 non-null  int64 
 2   word_count       17307 non-null  int64 
 3   essay_length     17307 non-null  int64 
 4   sentences_count  17307 non-null  int64 
 5   paragraph_count  17307 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 811.4+ KB


In [71]:
df_train

,full_text,score,word_count,essay_length,sentences_count,paragraph_count
0,Many people have car where they live. The thin...,3,498,2677,13,1
1,I am a scientist at NASA that is discussing th...,3,332,1669,19,9
2,People always wish they had the same technolog...,4,550,3077,24,7
3,"We all heard about Venus, the planet without a...",4,451,2701,23,9
4,"Dear, State Senator\n\nThis is a letter to arg...",3,373,2208,15,11
...,...,...,...,...,...,...
17302,"the story "" The Challenge of Exploing Venus "" ...",2,157,856,9,5
17303,Technology has changed a lot of ways that we l...,4,579,3354,28,11
17304,If you don't like sitting around all day than ...,2,215,1122,15,5
17305,"In ""The Challenge of Exporing Venus,"" the auth...",1,231,1427,11,1


In [72]:
df_train['full_text'][0]

'Many people have car where they live. The thing they don\'t know is that when you use a car alot of thing can happen\xa0like you can get in accidet or\xa0the smoke that the car has is bad to breath\xa0on if someone is walk but in VAUBAN,Germany they dont have that proble because 70 percent of vauban\'s families do not own cars,and 57 percent sold a car to move there. Street parkig ,driveways and home garages are forbidden\xa0on the outskirts of freiburd that near the French and Swiss borders. You probaly won\'t see a car in Vauban\'s streets because they are completely "car free" but\xa0If some that lives in VAUBAN that owns a car ownership is allowed,but there are only two places that you can park a large garages at the edge of the development,where a car owner buys a space but it not cheap to buy one they sell the space for you car for $40,000 along with a home. The vauban people completed this in 2006 ,they said that this an example of a growing trend in Europe,The untile states an

In [73]:
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)

In [74]:
def dataPreprocessing(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

In [75]:
def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    df['word_count'] = df['full_text'].apply(lambda x: len(x.split()))

    # Length
    df['essay_length'] = df['full_text'].str.len()

    # Sentences count
    # Adding a new column 'sentences_count' that counts the sentences in 'full_text'
    df['sentences_count'] = df['full_text'].str.count(r'\.')

    # Paragraph count
    # Adding a new column 'paragraph_count' that counts the paragraphs in 'full_text'
    df['paragraph_count'] = df['full_text'].str.count(r'\n') + 1
    
    df["text_tokens"] = df["full_text"].apply(lambda x: word_tokenize(x))
    df["word_count"] = df["text_tokens"].apply(lambda x: len(x))
    df["unique_word_count"] = df["text_tokens"].apply(lambda x: len(set(x)))

    df["processed_text"] = df["full_text"].apply(lambda x: dataPreprocessing(x))
    df["text_tokens"] = df["processed_text"].apply(lambda x: word_tokenize(x))
    df["text_length_p"] = df["processed_text"].apply(lambda x: len(x))
    df["word_count_p"] = df["text_tokens"].apply(lambda x: len(x))
    df["unique_word_count_p"] = df["text_tokens"].apply(lambda x: len(set(x)))
    
    df.drop(columns=["processed_text", "text_tokens"], inplace=True)
    return df

In [76]:
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Loại bỏ xuống dòng
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ ký tự đặc biệt
    text = re.sub(r'\xa0', ' ', text)  # Chuyển thành chữ thường
    return text

In [77]:
# df_train = preprocess_df(df_train)
df_train['clean_text'] = df_train['full_text'].apply(clean_text)
df_train = df_train.drop('full_text', axis = 1)

In [78]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   score            17307 non-null  int64 
 1   word_count       17307 non-null  int64 
 2   essay_length     17307 non-null  int64 
 3   sentences_count  17307 non-null  int64 
 4   paragraph_count  17307 non-null  int64 
 5   clean_text       17307 non-null  object
dtypes: int64(5), object(1)
memory usage: 811.4+ KB


In [79]:
df_train['clean_text'][0]

'Many people have car where they live The thing they dont know is that when you use a car alot of thing can happen like you can get in accidet or the smoke that the car has is bad to breath on if someone is walk but in VAUBANGermany they dont have that proble because 70 percent of vaubans families do not own carsand 57 percent sold a car to move there Street parkig driveways and home garages are forbidden on the outskirts of freiburd that near the French and Swiss borders You probaly wont see a car in Vaubans streets because they are completely car free but If some that lives in VAUBAN that owns a car ownership is allowedbut there are only two places that you can park a large garages at the edge of the developmentwhere a car owner buys a space but it not cheap to buy one they sell the space for you car for 40000 along with a home The vauban people completed this in 2006 they said that this an example of a growing trend in EuropeThe untile states and some where else are suburban life fr